In [1]:
pip install selenium==4.10.0

  Using cached selenium-4.10.0-py3-none-any.whl (6.7 MB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.0
    Uninstalling selenium-4.8.0:
      Successfully uninstalled selenium-4.8.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time 
import datetime
import pandas as pd

url = "https://twitter.com"

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

driver.get(url)

driver.implicitly_wait(3) 

# 로그인
login_button = driver.find_element(By.XPATH, "//span[contains(text(), '로그인')]")
login_button.click()

#아이디 비번 입력
with open(r"C:\Users\Gyoungwon Jeong\Desktop\sol_project_4\10. data crawling\twitter password.txt", "r") as twitter_login:
    lines = twitter_login.readlines()
    id = lines[0].strip()
    pw = lines[1].strip()
    
input_id = driver.find_element(By.XPATH, "//input[@name='text']") 
input_id.send_keys(id)
input_id.send_keys(Keys.ENTER)

input_pw = driver.find_element(By.XPATH, "//input[@name='password']")
input_pw.send_keys(pw)
input_pw.send_keys(Keys.ENTER)

# 최신 트렌드 가져오기
trends_for_you = driver.find_element(By.XPATH, "//div[@aria-label='Timeline: Trending now']")
trend_for_you = trends_for_you.find_elements(By.XPATH, ".//span[contains(@class, 'css-901oao') and not(contains(text(), 'Trends')) and not(contains(text(), 'posts')) and not(contains(text(), 'Trending')) and not(contains(text(), 'Show'))]")

trends = []

for trend in trend_for_you:
    trends.append(trend.text)

print(trends)

#검색 쿼리 가져오기
search_queries = []

today_date = datetime.datetime.now()
yesterday_date = today_date - datetime.timedelta(days=1)

start_date = yesterday_date.strftime("%Y-%m-%d")
end_date = today_date.strftime("%Y-%m-%d") 

for trend in trends:
    search_query = trend + ' lang:ko ' + 'until:' + end_date + ' since:' + start_date
    search_queries.append(search_query)

print(search_queries)

#검색어에 트렌드 검색
input_search = driver.find_element(By.XPATH, "//input[@aria-label='Search query']")
input_search.send_keys(search_queries[0])
input_search.send_keys(Keys.ENTER) 

seen_tweets = set()
tweet = []

while True:
    current_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #현재 스크롤 높이
    tweet_elements = driver.find_elements(By.XPATH, "//div[@class='css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0' and @data-testid='tweetText']")
    
    for tweet_element in tweet_elements:
        tweet_content = tweet_element.text.strip()
        if tweet_content not in seen_tweets:
            tweet.append(tweet_content) #현재 페이지 트윗 
            seen_tweets.add(tweet_content) #축적 트윗
    
    #print("Current Page Tweets Count:", len(tweet))
    #print("Seen Tweets Count:", len(seen_tweets))
    
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") #스크롤 내리기
    time.sleep(3)  # 콘텐츠 로딩 기다리기
    new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤 내린 후 높이
    if new_scroll_height == current_scroll_height:
        break

#트렌드 계속 하나씩 수집하기
for search_query in search_queries[1:]:
    input_search = driver.find_element(By.XPATH, "//input[@data-testid='SearchBox_Search_Input']")
    # 검색 창 내용을 선택하고 지우기
    input_search.send_keys(Keys.CONTROL + "a")
    input_search.send_keys(Keys.DELETE)
    input_search.send_keys(search_query)
    input_search.send_keys(Keys.ENTER)
    
    while True:
        current_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #현재 스크롤 높이
        tweet_elements = driver.find_elements(By.XPATH, "//div[@class='css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0' and @data-testid='tweetText']")
        
        for tweet_element in tweet_elements:
            tweet_content = tweet_element.text.strip()
            if tweet_content not in seen_tweets:
                tweet.append(tweet_content) #현재 페이지 트윗 
                seen_tweets.add(tweet_content) #축적 트윗
        
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") #스크롤 내리기
        time.sleep(3)  # 콘텐츠 로딩 기다리기
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤 내린 후 높이
        if new_scroll_height == current_scroll_height:
            break
    
# 데이터프레임에 넣기
twitter_df = pd.DataFrame({'content':tweet})
print(twitter_df)

['오토바이 사고', '남성과 연애함', '응원봉 품절', '악플 체험', '우주먼지', '오펜하이머', '자궁경부암', '태일오빠', '동양미래대', '주폭소년단']
['오토바이 사고 lang:ko until:2023-08-15 since:2023-08-14', '남성과 연애함 lang:ko until:2023-08-15 since:2023-08-14', '응원봉 품절 lang:ko until:2023-08-15 since:2023-08-14', '악플 체험 lang:ko until:2023-08-15 since:2023-08-14', '우주먼지 lang:ko until:2023-08-15 since:2023-08-14', '오펜하이머 lang:ko until:2023-08-15 since:2023-08-14', '자궁경부암 lang:ko until:2023-08-15 since:2023-08-14', '태일오빠 lang:ko until:2023-08-15 since:2023-08-14', '동양미래대 lang:ko until:2023-08-15 since:2023-08-14', '주폭소년단 lang:ko until:2023-08-15 since:2023-08-14']
Empty DataFrame
Columns: [content]
Index: []


In [16]:
twitter_df.to_csv('확인용2')